In [1]:
%load_ext autoreload
%autoreload 2
import re
import numpy as np
import tensorflow as tf
import os
import sys
from tf2gpt.model import GPT
from utils.story_util import Story,Stories
from utils.progress_bar import ProgressBar
from tensorboardX import SummaryWriter
from tensorflow.keras.utils import multi_gpu_model
import random

In [2]:
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:XLA_CPU:0', device_type='XLA_CPU'),
 PhysicalDevice(name='/physical_device:XLA_GPU:0', device_type='XLA_GPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


In [4]:
tf.keras.backend.set_floatx('float16')

In [5]:
#mirrored_strategy = tf.distribute.MirroredStrategy()

In [6]:
#with mirrored_strategy.scope():
gpt = GPT(
    vocab_size=30_000,
    layer_size=32,
    block_size=1024,
    embedding_dropout=0.0,
    embedding_size=2560,
    num_attention_heads=32,
    attention_dropout=0.0,
    residual_dropout=0.0,
    train_size=499
)
gpt.load_weights('./gpt_weight_pretrain/weight_fp16_pretrained_norm')

#input_x = tf.keras.layers.Input((499,), dtype=tf.int32)
#outputs = gpt_origin(input_x)

#gpt = tf.keras.Model(inputs=input_x, outputs=outputs)
#gpt = multi_gpu_model(gpt, gpus=8)

print(tf.keras.backend.floatx(), tf.float16, tf.keras.backend.floatx() == tf.float16)
if tf.keras.backend.floatx() == tf.float16:
    for x in gpt.weights:
        assert x.dtype == tf.float16


gpt.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),  # Optimizer
    # Loss function to minimize
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    # List of metrics to monitor
)

float16 <dtype: 'float16'> True


In [7]:
from utils.gpt2_tokenizer import GPT2Tokenizer
cbpe = GPT2Tokenizer(
    'CPM-Generate/bpe_3w_new/vocab.json',
    'CPM-Generate/bpe_3w_new/merges.txt',
    model_file='CPM-Generate/bpe_3w_new/chinese_vocab.model')

In [8]:
ids = cbpe.encode('今天天气还行')
ids

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.573 seconds.
Prefix dict has been built successfully.


[837, 259, 497, 57, 8, 237]

In [9]:
def test_basic_logic():
    ids = cbpe.encode('今天天气还行')
    print(ids)
    print("+" * 20)
    for i in range(10):
        output = gpt(tf.constant([ids]))
        print(output[0].shape)
        nid = np.argmax(output[0, -1])
        ids += [nid]
        print(i, cbpe.decode(ids))
        print(np.argmax(output[0],axis=-1))
        print(cbpe.decode(np.argmax(output[0],axis=-1)))
        print('-' * 30)
test_basic_logic()

[837, 259, 497, 57, 8, 237]
++++++++++++++++++++
(6, 30000)
0 今天天气 还 行 
[    8   497    46   788 25753     8]
气 很 不错暖和 
------------------------------
(7, 30000)
1 今天天气 还 行 ,
[    8   497    46   788 25753     8     9]
气 很 不错暖和 ,
------------------------------
(8, 30000)
2 今天天气 还 行 , 但
[    8   497    46   788 25753     8     9    51]
气 很 不错暖和 , 但
------------------------------
(9, 30000)
3 今天天气 还 行 , 但是
[    8   497    46   788 25753     8     9    51    35]
气 很 不错暖和 , 但是
------------------------------
(10, 30000)
4 今天天气 还 行 , 但是 我
[    8   497    46   788 25753     8     9    51    35    16]
气 很 不错暖和 , 但是 我
------------------------------
(11, 30000)
5 今天天气 还 行 , 但是 我 不
[    8   497    46   788 25753     8     9    51    35    16    24]
气 很 不错暖和 , 但是 我 不
------------------------------
(12, 30000)
6 今天天气 还 行 , 但是 我 不想
[    8   497    46   788 25753     8     9    51    35    16    24   404]
气 很 不错暖和 , 但是 我 不想
------------------------------
(13, 30000)
7 今天天气 还 行 , 但是 我 不想 在
[    8   49

In [10]:
def get_learning_rate(learning_rate=6e-4,
                      warmup_steps=20_0000,
                      decay_steps=200_0000,
                      alpha=0.0):
    def decayed_learning_rate(step=1):
        if step <= warmup_steps:
            mult = step / float(warmup_steps)
        else:
            progress = (step - warmup_steps) / (decay_steps - warmup_steps)
            mult = 0.5 * (1 + math.cos(math.pi * progress))
            mult = max(0.1, mult)
        return learning_rate * mult
    return decayed_learning_rate

In [11]:
stories = Stories("./labeled_data/advanture_translated/processed_translated_story.txt").stories
#stories = stories[:50]
data_folder = "./labeled_data/"
txt_files = [(data_folder + i) for i in os.listdir(data_folder) if "txt" in i]
#stories = stories[:10]
stories += [Story("","").from_file(i) for i in txt_files]

In [12]:
valid_stories = Stories("./labeled_data/advanture_translated/processed_translated_story_valid.txt").stories

In [13]:
len(stories),len(valid_stories)

(308, 35)

In [14]:
import copy
def data_generator(stories, batch_size=4,sample_len=200,inf=False):
    while True:
        batch_data = []
        tmp_stories = copy.copy(stories)
        random.shuffle(tmp_stories)
        for i,one_story in enumerate(tmp_stories):
            story_content = one_story.to_dungeon_format()
            story_content = story_content.replace("<start>\n","")
            story_content = story_content.replace("\n<end>","")
            story_content = story_content.replace("\n<end>","")
            story_content = story_content.replace(" ","")
            ids = cbpe.encode(story_content)
            while ids:
                sample = ids[:sample_len]
                ids = ids[sample_len:]
                if len(sample) < sample_len:
                    sample += [0 for i in range((sample_len - len(sample)))]
                batch_data.append(sample)
                if len(batch_data) >= batch_size:
                    yield np.asarray(batch_data)
                    batch_data = []
        if not inf:
            break

In [15]:
valid_gen = data_generator(valid_stories,sample_len=500,batch_size=2,inf=True)

In [16]:
valid_gen.__next__().shape

(2, 500)

In [ ]:
writer = SummaryWriter('log/finetune')
n_iter = 0
train_loss = 100
val_loss = 100
for epoch in range(200):
    print(f"Epoch {epoch + 1}")
    pb = ProgressBar(10000)
    pb.startjob()
    for x in data_generator(stories,sample_len=500,batch_size=2):
    #for x in data_generator_content(texts[508:],sample_len=400,batch_size=2):
        n_iter += 1
        ret = gpt.train_step(
        (
            tf.constant(x[:,:-1]),
            tf.constant(x[:,1:]))
        )
        writer.add_scalar("train/loss", ret["loss"].numpy(), n_iter)
        train_loss = ret["loss"].numpy()
        if n_iter % 10 == 0:
            valid_x = valid_gen.__next__()
            ret = gpt.eval_step(
            (
                tf.constant(valid_x[:,:-1]),
                tf.constant(valid_x[:,1:]))
            )
            writer.add_scalar("test/loss", ret["loss"].numpy(), n_iter)
            val_loss = ret["loss"].numpy()
        pb.info = f"tl: {train_loss} vl: {val_loss}"
        pb.complete(1)
    print()

Epoch 1
tl: 2.86328125 vl: 2.86328125 12.89 % [======>--------------------------------------------] 1289/10000 	 used:1444s eta:9759 ssss
Epoch 2
tl: 2.416015625 vl: 2.646484375 12.89 % [======>--------------------------------------------] 1289/10000 	 used:1443s eta:9756 ss
Epoch 3
tl: 2.744140625 vl: 2.828125 12.89 % [======>--------------------------------------------] 1289/10000 	 used:1443s eta:9755 s3 ss
Epoch 4
tl: 2.671875 vl: 1.515625 12.89 % [======>--------------------------------------------] 1289/10000 	 used:1443s eta:9757 s8 ss ss
Epoch 5
tl: 2.5546875 vl: 2.85546875 12.89 % [======>--------------------------------------------] 1289/10000 	 used:1444s eta:9762 s sss
Epoch 6
tl: 2.98046875 vl: 2.4921875 12.89 % [======>--------------------------------------------] 1289/10000 	 used:1447s eta:9779 ss ss
Epoch 7
tl: 3.123046875 vl: 2.357421875 12.89 % [======>--------------------------------------------] 1289/10000 	 used:1446s eta:9777 ss
Epoch 8
tl: 2.533203125 vl: 2.4570

In [ ]:
print(stories[0].to_dungeon_format())

# 使用

In [20]:
q = f'''你是一个公司老总，你事业正在上升期，你娶了一个美丽的妻子
> 你走进你的家门
'''
ids = cbpe.encode(q)
#print(ids)
#print("+" * 20)
for i in range(100):
    output = gpt(tf.constant([ids]))
    nid = np.argmax(output[0, -1])
    ids += [nid]
    print(i)
    
print(i,cbpe.decode(ids))

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
99 你 是 一个 公司老总 , 你 事业 正在 上升期 , 你 娶 了 一个 美丽 的 妻子 
 >   你 走进 你 的 家门 
 你 走进 你 的 家 , 你 的 妻子 正在 等 你 。 你 的 妻子 是 一个 美丽 的 女人 , 她 的 眼睛 是 蓝色 的 , 她 的 头发 是 棕色 的 , 她 的 皮肤 是 棕色 的 , 她 的 嘴唇 是 红色 的 , 她 的 鼻子 是 红色 的 , 她 的 嘴唇 是 红色 的 。 她 的 头发 是 棕色 的 , 她 的 眼睛 是 蓝色 的 , 她 的 皮肤 是


In [19]:
gpt.save_weights('./gpt_weight_pretrain/weight_fp16_200epoch_308stories_dungeonformat')

# save model

In [5]:
ids = cbpe.encode('今天天气不错')

for i in range(10):
    output = gpt(tf.constant([ids]))
    nid = np.argmax(output[0, -1])
    ids += [nid]
    print(i, cbpe.decode(ids))

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.560 seconds.
Prefix dict has been built successfully.


0 今天天气 不错 
1 今天天气 不错 ,
2 今天天气 不错 , 我
3 今天天气 不错 , 我 想
4 今天天气 不错 , 我 想 我
5 今天天气 不错 , 我 想 我 可以
6 今天天气 不错 , 我 想 我 可以 出去
7 今天天气 不错 , 我 想 我 可以 出去 
8 今天天气 不错 , 我 想 我 可以 出去 。
9 今天天气 不错 , 我 想 我 可以 出去 。 


In [6]:
def batch_gather(a, b):
    return tf.gather(a, b, batch_dims=1)


def top_p_sample(logits, num_samples=1, p=0.95):
    batch_size, vocab_size = logits.shape
    probs = tf.nn.softmax(logits, axis=-1)
    # [batch_size, vocab_perm]
    indices = tf.argsort(probs, direction='DESCENDING')
    cumulative_probabilities = tf.math.cumsum(batch_gather(probs, indices), axis=-1, exclusive=False)

    # find the top pth index to cut off. careful we don't want to cutoff everything!
    # result will be [batch_size, vocab_perm]
    p_expanded = p if isinstance(p, float) else p[:, None]
    exclude_mask = tf.logical_not(
        tf.logical_or(cumulative_probabilities < p_expanded, tf.range(vocab_size)[None] < 1))

    # OPTION A - sample in the sorted space, then unsort.
    logits_to_use = batch_gather(logits, indices) - tf.cast(exclude_mask, tf.float16) * 1e4
    sample_perm = tf.random.categorical(logits=logits_to_use, num_samples=num_samples)
    sample = batch_gather(indices, sample_perm)

    return tf.cast(sample, tf.int64)

In [7]:
@tf.function
def serve(inputs):
    return gpt(inputs, kv_cache=None, use_cache=True)


@tf.function
def serve_cache(inputs, kv_cache):
    return gpt(inputs, kv_cache=kv_cache, use_cache=True)

In [8]:
serve_concrete = serve.get_concrete_function(
    tf.TensorSpec(shape=[None, None], dtype=tf.int64, name="inp")
)

layer_size = 32
attention_head = 32
embedding_size = 2560

serve_cache_concrete = serve_cache.get_concrete_function(
    tf.TensorSpec(shape=[None, None], dtype=tf.int64, name="inp"),
    tf.TensorSpec(shape=[
        layer_size, None, 2, attention_head,
        None, embedding_size // attention_head
    ], dtype=tf.float16, name="kv_cache")
)

In [9]:
r = serve_concrete(
    tf.constant([[1]], tf.int64)
)
r2 = serve_cache_concrete(
    tf.constant([[1]], tf.int64),
    r[1]
)

In [10]:
@tf.function
def sample(initial_inputs, length):
    layer_size = 32
    embedding_size = 2560
    attention_head = 32

    i = tf.constant(0, dtype=tf.int64)
    initial_logits, kv_cache = serve(initial_inputs)
    inputs = top_p_sample(initial_logits[:, -1, :])
    stores = tf.concat([initial_inputs, inputs], axis=1)

    def _cond(i, inputs, kv_cache, stores):
        return i < length

    def _body(i, inputs, kv_cache, stores):
        new_logits, new_kv_cache = serve_cache(inputs, kv_cache)
        
        new_inputs = top_p_sample(new_logits[:, -1, :])
        new_stores = tf.concat([stores, new_inputs], axis=-1)
        new_kv_cache = tf.concat([
            kv_cache,
            new_kv_cache
        ], axis=-2)
        new_i = i + 1
        return [new_i, new_inputs, new_kv_cache, new_stores]

    result = tf.while_loop(
        _cond, _body,
        loop_vars=[i, inputs, kv_cache, stores],
        shape_invariants=[
            tf.TensorShape(None),
            tf.TensorShape([None, None]),
            tf.TensorShape([
                layer_size, None, 2,
                attention_head, None,
                embedding_size // attention_head
            ]),
            tf.TensorShape([
                None, None
            ])
        ]
    )
    return result[-1]

In [10]:
gpt.save('./result_weights/cpm-lm-tf2-fp16-fine4m-dungeon-format2day', include_optimizer=False, signatures={
    'serving_default': sample.get_concrete_function(
        tf.TensorSpec(shape=[None, None], dtype=tf.int64, name="inp"),
        tf.TensorSpec(shape=[None,], dtype=tf.int64, name="length")
    )
})

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: ./result_weights/cpm-lm-tf2-fp16-fine4m-dungeon-format2day/assets


In [21]:
ids = cbpe.encode('今天天气不错')

ret = sample(
    tf.constant([ids], dtype=tf.int64),
    tf.constant(10, dtype=tf.int64)
)
print(ret)
print(cbpe.decode(ret.numpy().tolist()[0]))

tf.Tensor(
[[  837   259   497   788     8     9   457    27     8 25427     8    12
      8    34     8]], shape=(1, 15), dtype=int64)
今天天气 不错 , 向 你 眨眼 。 ” 


# read deploy model (you should restart kernel and go from here)

In [1]:
%load_ext autoreload
%autoreload 2
import re
import numpy as np
import tensorflow as tf
import os
import sys
from tf2gpt.model import GPT
from utils.story_util import Story,Stories
from utils.progress_bar import ProgressBar
from tensorboardX import SummaryWriter
from tensorflow.keras.utils import multi_gpu_model
import random
from utils.gpt2_tokenizer import GPT2Tokenizer
import tensorflow_hub as hub
cbpe = GPT2Tokenizer(
    'CPM-Generate/bpe_3w_new/vocab.json',
    'CPM-Generate/bpe_3w_new/merges.txt',
    model_file='CPM-Generate/bpe_3w_new/chinese_vocab.model')

In [2]:
gpt = hub.load('./result_weights/cpm-lm-tf2-fp16-fine4m-dungeon-format2day')

In [3]:
def sample_gpt(tokenizer, gpt, sentence, number=1, length=20):
    inputs = tf.constant([tokenizer.encode(sentence)] * number, dtype=tf.int64)
    length = tf.constant(length, dtype=tf.int64)
    ret = gpt.signatures['serving_default'](inp=inputs, length=length)['output_0']
    return [
        tokenizer.decode(s).replace(' ', '')
        for s in ret.numpy()
    ]

In [4]:
q = f'''你是一个公司老总，你事业正在上升期，你娶了一个美丽的妻子
> 你走进你的家门
'''
ret = sample_gpt(cbpe, gpt, q, 1, 150)
for x in ret:
    print(x)
    print('-' * 20)

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.676 seconds.
Prefix dict has been built successfully.


你是一个公司老总,你事业正在上升期,你娶了一个美丽的妻子
>你走进你的家门
玛丽,埃莉诺,孩子和他们的朋友走近你。当他们到达的时候,他们开始哄笑起来。“爸爸,妈妈,我们能和你玩吗?”玛丽发出了命令。你点点头,说:“当然,好的。当我和你们一起去骑马时,我想我会用这只手把它们从鞍子上拿开。”
>你骑马去看
当其他的孩子都加入战斗,你骑着你的骏马穿过城镇,将两个时代分开。当
--------------------


In [8]:
class Story():
    def __init__(self,beginning,story_max_len=150,context_len=6):
        self.story = [beginning]
        self.story_max_len = story_max_len
        self.context_len = context_len
    
    def action(self,action):
        action_str = "> 你" + action
        self.story.append(action_str)
        q = "\n".join(self.story[-self.context_len:])
        response = sample_gpt(cbpe, gpt, q, 1, 150)[0][len(q):]
        self.story.append(response)
        
    def interactive(self):
        print("\n".join(self.story))
        while True:
            action = input("> 你")
            self.action(action)
            print()
            print(self.story[-1])

In [9]:
begin = f'''你在树林里冒险，指不定会从哪里蹦出来一些奇怪的东西，你握紧手上的手枪，希望这次冒险能够找到一些值钱的东西'''
story = Story(begin)

In [ ]:
story.interactive()

你在树林里冒险，指不定会从哪里蹦出来一些奇怪的东西，你握紧手上的手枪，希望这次冒险能够找到一些值钱的东西


> 你 向树林深处走去



又走了好一会儿,天空中闪烁着明亮的光,可以让您看到厚厚的森林。森林真的很茂密,简直盖过了一座小山。光线从树叶的缝隙中穿过,洒在到处都是苔藓的地面上。一块足够小的地方可以放下你的任何东西,这里只有一个入口。
>你向前方跑去
你又向前跑了一段路,接着就穿过了一排排小房子,进入到一个闪烁着明亮光的新绿之中。森林的最北端是一片绿色的山丘


> 你 走上绿色的山丘



!这是一个尽头!你超越了上方的树枝,开始迈开两条腿走。远处的太阳照在地面上,你们能看到下面的大地和蓝天。你带着一股干净的气息站在他们面前,在它温暖的空气中嗅着它的芳香。你四处走动,寻找一些舒服的地方,然后试图将注意力集中在工作上。你在这里感觉就像是重新开始,并没有多么不适合那个在遭遇那些黑暗森林居民的泥土之前一直被囚禁的地方。你决定冒险进入他们的地区,在他


...


说它是什么,小丑?”你朝它靠近,怀疑地收起了刚才在本能下可能看到的手臂和腿。“它是一个女人,”你说,举起了一把匕首,“她在这里,你看看。我不想这么做,因为它看起来太令人惊讶了。我不应该被她占据时间,更不想做一个猎物。”你听到她咯咯的笑声,当你凝视她时,她将笑声投向你。你张开嘴巴,慢慢地张开


> 你 问她是什么人



那一刻真的是在搞笑,在你以一种痛苦的好奇心看着她。然后你开始享受其中的乐趣,转向马,它已经开始跑向你。几分钟后,尽管你觉得有点累,你相信小丑真的存在。马身细长,可是每个部分都小得可怜,边跑边拉,旋转,骑,边撒吃的。你骑着它飞驰而去,飞向远方。你回头看了看,小丑仍然在他的阴影中闲逛,眼睛在悄悄望着什么。你环顾四


> 你 ok你到底是谁



知不觉已经跨过了地雷区,越过了叛徒。当你猛踩加速时,小丑举起了剑。你将匕首刺入他。一阵风,有微风和尘土吹开了他,时间似乎静止了。你们在一棵大树下面,“蒂德,我来消灭你。我在树林里长大,是个德鲁伊,这是我的故事。我是人类,是个性感玛丽。我只想知道自己是谁,想寻找自己的生活。但你知道,这并不重要。
